# Cisco Sample Code
## 102 REST

In [1]:
import requests
import json
from helper.spark import *

requests.packages.urllib3.disable_warnings()

controller='sandboxapicem.cisco.com'
username = "devnetuser"
password = "Cisco123!"
version = "v1"

## 通用函数

In [2]:
def getTicket():
    # put the ip address or dns of your apic-em controller in this url
    url = "https://" + controller + "/api/v1/ticket"

    #the username and password to access the APIC-EM Controller    
    payload = {"username":username,"password":password}

    #Content type must be included in the header
    header = {"content-type": "application/json"}

    #Performs a POST on the specified url to get the service ticket
    response= requests.post(url,data=json.dumps(payload), headers=header, verify=False)

    print (response)
    
    #convert response to json format
    r_json=response.json()

    #parse the json to get the service ticket
    ticket = r_json["response"]["serviceTicket"]

    return ticket

### Build Topology

In [7]:
def getTopology():
    ticket=getTicket()
    
    # URL for topology REST API call to get list of existing devices on the network, and build topology
    url = "https://" + controller + "/api/v1/topology/physical-topology"

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}

    # this statement performs a GET on the specified network device url
    response = requests.get(url, headers=header, verify=False)

    # json.dumps serializes the json into a string and allows us to
    # print the response in a 'pretty' format with indentation etc.
    print ("Topology = ")
    print (json.dumps(response.json(), indent=4, separators=(',', ': ')))
    
    #convert data to json format.
    r_json=response.json()
    
    #Iterate through network device data and list the nodes, their interfaces, status and to what they connect    
    for n in r_json["response"]["nodes"]:
        if "platformId" in n:
            print()
            print()
            print('{:30}'.format("Node") + '{:25}'.format("Family") + '{:20}'.format("Label")+ "Management IP")
            print('{:30}'.format(n["platformId"]) + '{:25}'.format(n["family"]) + '{:20}'.format(n["label"]) + n["ip"])        
        found=0    #print header flag
        printed=0  #formatting flag
        for i in r_json["response"]["links"]:
            if "startPortName" in i:
                #check that the source device id for the interface matches the node id.  Means interface originated from this device. 
                if i["source"] == n["id"]:
                    if found==0:
                        print('{:>20}'.format("Source Interface") + '{:>15}'.format("Target") +'{:>28}'.format("Target Interface") + '{:>15}'.format("Status") )
                        found=1
                        printed=1                    
                    for n1 in r_json["response"]["nodes"]:
                        #find name of node to which this one connects
                        if i["target"] == n1["id"]:
                            print("    " + '{:<25}'.format(i["startPortName"]) + '{:<18}'.format(n1["platformId"]) + '{:<25}'.format(i["endPortName"]) + '{:<9}'.format(i["linkStatus"]) )                            
                            break;
        found=0        
        
        for i in r_json["response"]["links"]:
            if "startPortName" in i:
                #Find interfaces that link to this one which means this node is the target. 
                if i["target"] == n["id"]:
                    if found==0:
                        if printed==1:
                            print()
                        print('{:>10}'.format("Source") + '{:>30}'.format("Source Interface") + '{:>25}'.format("Target Interface") + '{:>13}'.format("Status"))
                        found=1                    
                    for n1 in r_json["response"]["nodes"]:
                        #find name of node to that connects to this one
                        if i["source"] == n1["id"]:                            
                            print("    " + '{:<20}'.format(n1["platformId"]) + '{:<25}'.format(i["startPortName"]) + '{:<23}'.format(i["endPortName"]) + '{:<8}'.format(i["linkStatus"]))
                            break;
        

getTopology()

<Response [200]>
Topology = 
{
    "response": {
        "nodes": [
            {
                "deviceType": "Cisco 2911 Integrated Services Router G2",
                "label": "Branch-Router1",
                "ip": "10.2.2.1",
                "softwareVersion": "15.2(4)M6a",
                "nodeType": "device",
                "family": "Routers",
                "platformId": "CISCO2911/K9",
                "tags": [
                    "test3"
                ],
                "role": "BORDER ROUTER",
                "roleSource": "MANUAL",
                "customParam": {},
                "id": "0dd240fd-5cca-4774-a801-9f1c04edcc70"
            },
            {
                "deviceType": "Cisco Catalyst 6503 Switch",
                "label": "CAMPUS-Core2",
                "ip": "10.1.10.1",
                "softwareVersion": "15.2(1)SY1",
                "nodeType": "device",
                "family": "Switches and Hubs",
                "platformId": "WS-C6503-E",
    

### Get Network Devices

In [12]:
def getNetworkDevices():
    
    ticket=getTicket()
    
    # URL for network-device REST API call to get list of exisiting devices on the network.
    url = "https://" + controller + "/api/v1/network-device"

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}

    # this statement performs a GET on the specified network device url
    response = requests.get(url, headers=header, verify=False)

    # json.dumps serializes the json into a string and allows us to
    # print the response in a 'pretty' format with indentation etc.
    print ("Network Devices = ")
    print (json.dumps(response.json(), indent=4, separators=(',', ': ')))
    
    #convert data to json format.
    r_json=response.json()
    
    #Iterate through network device data and print the id and series name of each device
    for i in r_json["response"]:
        print(i["id"] + "   " + '{:53}'.format(i["series"]) + "  " + i["reachabilityStatus"])
        
    return r_json, response
        

r_json, response = getNetworkDevices()

<Response [200]>
Network Devices = 
{
    "response": [
        {
            "errorCode": "DEV-UNREACHED",
            "serialNumber": "FOC1637Y3FJ",
            "macAddress": "54:78:1a:8e:28:c0",
            "family": "Switches and Hubs",
            "type": "Cisco Catalyst 2960C-8PC-L Switch",
            "location": "df8a7d62-a156-4fcd-8573-cc1421f7a14d",
            "inventoryStatusDetail": "<status><general code=\"DEV_UNREACHED\"/></status>",
            "upTime": "16:11:38.75",
            "role": "DISTRIBUTION",
            "lastUpdated": "2020-02-22 06:17:05",
            "softwareVersion": "15.2(2.5.72)E4",
            "lastUpdateTime": 1582352225100,
            "locationName": "Riyadh",
            "errorDescription": "SNMP timeouts are occurring with this device. Either the SNMP credentials are not correctly provided to controller or the device is responding too slow. Please ensure correct credentials are provided in discovery/global credentials and run discovery again.  Y

### Get Network Hosts

In [30]:
def getNetworkHosts():
    ticket=getTicket()
    url = "https://" + controller + "/api/v1/host?limit=1&offset=1"
    header = {"content-type": "application/json", "X-Auth-Token":ticket}
    response = requests.get(url, headers=header, verify=False)
    r_json = response.json()
    print ("Hosts = ")
    print (json.dumps(r_json, indent=4, separators=(',', ': ')))
    
getNetworkHosts()

<Response [200]>
Hosts = 
{
    "response": [
        {
            "hostIp": "10.1.15.117",
            "hostMac": "00:24:d7:43:59:d8",
            "hostType": "wireless",
            "connectedNetworkDeviceId": "cd6d9b24-839b-4d58-adfe-3fdf781e1782",
            "connectedNetworkDeviceIpAddress": "10.1.14.3",
            "connectedAPMacAddress": "68:bc:0c:63:4a:b0",
            "connectedAPName": "AP7081.059f.19ca",
            "vlanId": "600",
            "lastUpdated": "1479514114932",
            "source": "200",
            "pointOfPresence": "ae19cd21-1b26-4f58-8ccd-d265deabb6c3",
            "pointOfAttachment": "ae19cd21-1b26-4f58-8ccd-d265deabb6c3",
            "subType": "UNKNOWN",
            "id": "48cdeb9b-b412-491e-a80c-7ec5bbe98167"
        }
    ],
    "version": "1.0"
}


### Manage Users

In [31]:
def getUsers(ticket):
    print("\nGetting list of existing users")
    # URL for user REST API call to get list of APIC-EM users.
    url = "https://" + controller + "/api/v1/user"

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}

    # this statement performs a GET on the specified host url
    response = requests.get(url, headers=header, verify=False)

    # json.dumps serializes the json into a string and allows us to
    # print the response in a 'pretty' format with indentation etc.
    print ("Users = ")
    print (json.dumps(response.json(), indent=4, separators=(',', ': ')))

    
#Adds a APIC-EM User
def addUser(ticket):
    print("\nAdding new user")
    # URL for user REST API call to get list of existing users in the network.
    url = "https://" + controller + "/api/v1/user"

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}
    
    username="brett"
    #Data for new user
    payload={"password":"Brett123!","username":username,"authorization":[{"scope":"ALL","role":"ROLE_OBSERVER"}]}
    
    # this statement performs a Post on the specified user url
    response = requests.post(url, data=json.dumps(payload), headers=header, verify=False)
    print ("Response after post:  " + response.text)
    
    return (username)


#Delete the user that corresponds to the passed in username parameter
def deleteUser(username, ticket):
    print("\nRemoving user: " + username)
    # URL for a specified user REST API call.
    url = "https://" + controller + "/api/v1/user/" + username

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}

    # this statement performs a Delete on the specified user url
    response = requests.delete(url, headers=header, verify=False)
    print (response.text)

    
#Show the User that corresponds to the passed in username parameter
def showUser(username, ticket):
    print("\nDisplaying user: " + username)
    # URL for user REST API call to get APIC-EM user with corresponding name.
    url = "https://" + controller + "/api/v1/user/" + username

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}

    # this statement performs a GET on the specified user url
    response = requests.get(url, headers=header, verify=False)
    
    # json.dumps serializes the json into a string and allows us to
    # print the response in a 'pretty' format with indentation etc.
    print ("User found = ")
    print (json.dumps(response.json(), indent=4, separators=(',', ': ')))
    


theTicket=getTicket()
getUsers(theTicket)
name=addUser(theTicket)
showUser(name,theTicket)
getUsers(theTicket)
deleteUser(name,theTicket)
getUsers(theTicket)

<Response [200]>

Getting list of existing users
Users = 
{
    "response": [
        {
            "username": "devnetuser",
            "authorization": [
                {
                    "scope": "ALL",
                    "role": "ROLE_POLICY_ADMIN"
                }
            ],
            "authSource": "internal"
        }
    ],
    "version": "1.0"
}

Adding new user
Response after post:  { "response":{"errorCode":"RBAC","message":"You do not have permissions to access this resource","detail":"You do not have permissions to access this resource"}, "version":"1.0"}


Displaying user: brett
User found = 
{
    "response": {
        "errorCode": "Unauthorized",
        "message": "You do not have permissions to access this resource",
        "detail": "User does not have sufficient privileges to access API"
    },
    "version": "1.0"
}

Getting list of existing users
Users = 
{
    "response": [
        {
            "username": "devnetuser",
            "authorization": 

### Report Topology

In [3]:
def getTopology():
    
    ticket=getTicket()
    # URL for topology REST API call to get list of existing devices on the network, and build topology
    url = "https://" + controller + "/api/v1/topology/physical-topology"

    #Content type as well as the ticket must be included in the header 
    header = {"content-type": "application/json", "X-Auth-Token":ticket}

    # this statement performs a GET on the specified network device url
    response = requests.get(url, headers=header, verify=False)

    # json.dumps serializes the json into a string and allows us to
    # print the response in a 'pretty' format with indentation etc.
    print ("Topology = ")
    print (json.dumps(response.json(), indent=4, separators=(',', ': ')))
    
    #convert data to json format.
    r_json=response.json()
    
    net_nodes=[]
    #Iterate through network device data and list the nodes, their interfaces, status and to what they connect    
    for n in r_json["response"]["nodes"]:
        if "platformId" in n:
            print()
            print()
            print('{:30}'.format("Node") + '{:25}'.format("Family") + '{:20}'.format("Label")+ "Management IP")
            print('{:30}'.format(n["platformId"]) + '{:25}'.format(n["family"]) + '{:20}'.format(n["label"]) + n["ip"])
            net_nodes.append(n["label"])
            
        found=0    #print header flag        
        for i in r_json["response"]["links"]:
            if "startPortName" in i:
                #check that the source device id for the interface matches the node id.  Means interface originated from this device. 
                if i["source"] == n["id"]:
                    if found==0:
                        print('{:>20}'.format("Source Interface") + '{:>15}'.format("Target") +'{:>28}'.format("Target Interface") + '{:>15}'.format("Status") )
                        found=1                                            
                    for n1 in r_json["response"]["nodes"]:
                        #find name of node to which this one connects
                        if i["target"] == n1["id"]:
                            print("    " + '{:<25}'.format(i["startPortName"]) + '{:<18}'.format(n1["platformId"]) + '{:<25}'.format(i["endPortName"]) + '{:<9}'.format(i["linkStatus"]) )                            
                            break
        
    return net_nodes
    
    
theNodes=getTopology()
nodestr=""
for node in theNodes:
    nodestr=nodestr + node + " "
setHeaders()
room_id=createRoom("Brett's Room")
addMembers(room_id)   # Passing roomId to members function here to Post Message.
postMsg(room_id,"Network nodes are " + nodestr)      # Passing roomId to message function here to Post Message.

<Response [200]>
Topology = 
{
    "response": {
        "nodes": [
            {
                "deviceType": "Cisco 2911 Integrated Services Router G2",
                "label": "Branch-Router1",
                "ip": "10.2.2.1",
                "softwareVersion": "15.2(4)M6a",
                "nodeType": "device",
                "family": "Routers",
                "platformId": "CISCO2911/K9",
                "tags": [
                    "test3"
                ],
                "role": "BORDER ROUTER",
                "roleSource": "MANUAL",
                "customParam": {},
                "id": "0dd240fd-5cca-4774-a801-9f1c04edcc70"
            },
            {
                "deviceType": "Cisco Catalyst 6503 Switch",
                "label": "CAMPUS-Core2",
                "ip": "10.1.10.1",
                "softwareVersion": "15.2(1)SY1",
                "nodeType": "device",
                "family": "Switches and Hubs",
                "platformId": "WS-C6503-E",
    

SparkAPI:  <Response [401]>
{'message': 'The request requires a valid access token set in the Authorization request header.', 'errors': [{'description': 'The request requires a valid access token set in the Authorization request header.'}], 'trackingId': 'ROUTER_5E50CD9B-2A3A-01BB-00F7-7D4BA75D00F7'}


KeyError: 'id'